# Langchain4j LLM streaming

In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.36.2";
String langgraph4jVersion = "1.1-SNAPSHOT";

In [2]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Repositories count: 4
name: central url: https://repo.maven.apache.org/maven2/ release:true update:never snapshot:false update:never 
name: jboss url: https://repository.jboss.org/nexus/content/repositories/releases/ release:true update:never snapshot:false update:never 
name: atlassian url: https://packages.atlassian.com/maven/public release:true update:never snapshot:false update:never 
name: local url: file:///Users/bsorrentino/.m2/repository/ release:true update:never snapshot:true update:always 


Remove installed package from Jupiter cache

In [3]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/

In [4]:
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core-jdk8:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:\{langgraph4jVersion}
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core-jdk8:1.1-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.1-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:0.36.2
Adding dependency dev.langchain4j:langchain4j-open-ai:0.36.2
Solving dependencies
Resolved artifacts count: 26
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core-jdk8/1.1-SNAPSHOT/langgraph4j-core-jdk8-1.1-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/async/async-generator-jdk8/2.3.0/async-generator-jdk8-2.3.0.jar
Add 

### Initialize Logger

In [5]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    var lm = java.util.logging.LogManager.getLogManager();
    lm.checkAccess(); 
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("llm-streaming");


In [6]:
import dev.langchain4j.model.StreamingResponseHandler;
import dev.langchain4j.model.chat.StreamingChatLanguageModel;
import dev.langchain4j.model.openai.OpenAiStreamingChatModel;
import dev.langchain4j.data.message.AiMessage;
import dev.langchain4j.model.output.Response;
import static dev.langchain4j.model.openai.OpenAiChatModelName.GPT_4_O_MINI;
import org.bsc.langgraph4j.langchain4j.generators.LLMStreamingGenerator;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.streaming.StreamingOutput;

var generator = LLMStreamingGenerator.<AiMessage,AgentState>builder()
                        .mapResult( r -> Map.of( "content", r.content() ) )
                        .build();

StreamingChatLanguageModel model = OpenAiStreamingChatModel.builder()
    .apiKey(System.getenv("OPENAI_API_KEY"))
    .modelName(GPT_4_O_MINI)
    .build();

String userMessage = "Tell me a joke";

model.generate(userMessage, generator.handler() );

for( var r : generator ) {
    log.info( "{}", r);
}
  
log.info( "RESULT: {}", generator.resultValue().orElse(null) );
  
//Thread.sleep( 1000 );

StreamingOutput{chunk=} 
StreamingOutput{chunk=Why} 
StreamingOutput{chunk= did} 
StreamingOutput{chunk= the} 
StreamingOutput{chunk= scare} 
StreamingOutput{chunk=crow} 
StreamingOutput{chunk= win} 
StreamingOutput{chunk= an} 
StreamingOutput{chunk= award} 
StreamingOutput{chunk=?} 
StreamingOutput{chunk= 

} 
StreamingOutput{chunk=Because} 
StreamingOutput{chunk= he} 
StreamingOutput{chunk= was} 
StreamingOutput{chunk= outstanding} 
StreamingOutput{chunk= in} 
StreamingOutput{chunk= his} 
StreamingOutput{chunk= field} 
StreamingOutput{chunk=!} 
RESULT: {content=AiMessage { text = "Why did the scarecrow win an award? 

Because he was outstanding in his field!" toolExecutionRequests = null }} 
